# Computing PCA using RDDs

###  PCA

The vectors that we want to analyze have length, or dimension, of 365, corresponding to the number of 
days in a year.

We want to perform [Principle component analysis (PCA)](https://en.wikipedia.org/wiki/Principal_component_analysis)
on these vectors. There are two steps to this process:

1. Computing the covariance matrix: this is a  simple computation. However, it takes a long time to compute and it benefits from using an RDD because it involves all of the input vectors.
2. Computing the eigenvector decomposition. this is a more complex computation, but it takes a fraction of a second because the size to the covariance matrix is $365 \times 365$, which is quite small. We do it on the head node usin `linalg`

### Computing the covariance matrix
Suppose that the data vectors are the column vectors denoted $x$ then the covariance matrix is defined to be
$$
E(x x^T)-E(x)E(x)^T
$$

Where $x x^T$ is the **outer product** of $x$ with itself.

If the data that we have is $x_1,x_2,x_n$ then the estimates we use are:
$$
\hat{E}(x x^T) = \frac{1}{n} \sum_{i=1}^n x_i x_i^T,\;\;\;\;\;
\hat{E}(x) = \frac{1}{n} \sum_{i=1}^n x_i
$$

### `nan`s in arithmetic operations
* We use an RDD of numpy arrays, instead od Dataframes.
* Why? Because numpy treats `nan` entries correctly:
  * In numpy `5+nan=5` while in dataframes `5+nan=nan`

### Performing Cov matrix on vectors with NaNs
As it happens, we often get vectors $x$ in which some, but not all, of the entries are `nan`. In such cases we sum the elements that are defined and keep a seperate counter for each entry.

In [31]:
import numpy as np

def outerProduct(X):
  """Computer outer product and indicate which locations in matrix are undefined"""
  O=np.outer(X,X)
  N=1-np.isnan(O)
  return (O,N)
def sumWithNan(M1,M2):
  """Add two pairs of (matrix,count)"""
  (X1,N1)=M1
  (X2,N2)=M2
  N=N1+N2
  X=np.nansum(np.dstack((X1,X2)),axis=2)
  return (X,N)
  

In [32]:
# computeCov recieves as input an RDD of np arrays, all of the same length, and computes the covariance matrix for that set of vectors
def computeCov(RDDin):
  RDD=RDDin.map(lambda v:np.insert(v,0,1)) # insert a 1 at the beginning of each vector so that the same 
                                           #calculation also yields the mean vector
  OuterRDD=RDD.map(outerProduct)   # separating the map and the reduce does not matter because of Spark uses lazy execution.
  (S,N)=OuterRDD.reduce(sumWithNan)

  # Unpack result and compute the covariance matrix
  print 'shape of S=',S.shape,'shape of N=',N.shape
  E=S[0,1:]
  NE=np.float64(N[0,1:])
  print 'shape of E=',E.shape,'shape of NE=',NE.shape
  Mean=E/NE
  O=S[1:,1:]
  NO=np.float64(N[1:,1:])
  Cov=O/NO - np.outer(Mean,Mean)
  # Output also the diagnal which is the variance for each day
  Var=np.array([Cov[i,i] for i in range(Cov.shape[0])])
  return {'E':E,'NE':NE,'O':O,'NO':NO,'Cov':Cov,'Mean':Mean,'Var':Var}

## Reading from S3 using the spark-notebook server.
#### Set AWS Credentials

To access a S3 bucket, the first step is to set AWS credential. There are two ways to do it.

1. (**RECOMMENDED**) Set S3 credentials via Spark Notebook interface.
2. Set it using the `set_credential` method:
```python
s3helper.set_credential(AWS_ACCESS_KEY_ID, AWS_SECRET_ACCESS_KEY)
```

In [3]:
from pyspark import SparkContext

sc = SparkContext(master=master_url)

In [4]:
bucket='mas-dse-public'
dir='/home/ec2-user/spark/'
s3helper.open_bucket(bucket)
from time import time

In [ ]:
t=time()
files = s3helper.s3_to_hdfs('/Weather/US_Weather.parquet', '/US_Weather.parquet')
print 'copying took',time()-t,'seconds'
## copying took 62.695912838 seconds

In [61]:
print s3helper.ls_s3()  # By default, list all files in the root directory of the bucket
print s3helper.ls_s3('Weather')

print s3helper.ls_hdfs()

[u'Scripts', u'Scripts_$folder$', u'Spark-Data', u'Spark-Data_$folder$', u'Weather', u'Weather_$folder$', u'moby10b.txt']
[u'Weather/PCA_PRCP.pickle', u'Weather/PCA_SNOW.pickle', u'Weather/PCA_TMAX.pickle', u'Weather/STAT1.pickle', u'Weather/SampleStations.pickle', u'Weather/US_Weather.parquet', u'Weather/US_counts.pickle', u'Weather/Weather.parquet', u'Weather/Weather_Stations.parquet', u'Weather/test.json', u'Weather_$folder$']
Found 1 items
drwxr-xr-x   - ec2-user supergroup          0 2017-03-08 00:52 /US_Weather.parquet

None


In [6]:
from pyspark import SparkContext
from pyspark.sql import SQLContext

sqlContext = SQLContext(sc)

In [7]:
t=time()
TMAX = sqlContext.sql("SELECT * FROM parquet.`/US_Weather.parquet` where measurement = 'TMAX'")
print TMAX.count()
print 'took ',time()-t,' seconds'

662767
took  7.91540503502  seconds


## Computing the statistics

In [8]:
STAT_Descriptions=[
('SortedVals', 'Sample of values', 'vector whose length varies between measurements'),
 ('UnDef', 'sample of number of undefs per row', 'vector whose length varies between measurements'),
 ('mean', 'mean value', ()),
 ('std', 'std', ()),
 ('low100', 'bottom 1%', ()),
 ('high100', 'top 1%', ()),
 ('low1000', 'bottom 0.1%', ()),
 ('high1000', 'top 0.1%', ()),
 ('E', 'Sum of values per day', (365,)),
 ('NE', 'count of values per day', (365,)),
 ('Mean', 'E/NE', (365,)),
 ('O', 'Sum of outer products', (365, 365)),
 ('NO', 'counts for outer products', (365, 365)),
 ('Cov', 'O/NO', (365, 365)),
 ('Var', 'The variance per day = diagonal of Cov', (365,)),
 ('eigval', 'PCA eigen-values', (365,)),
 ('eigvec', 'PCA eigen-vectors', (365, 365))
  ]

In [11]:
US_Weather_parquet='/US_Weather.parquet'
measurements=['TMAX','TMIN','TOBS','SNOW','SNWD','PRCP']
Query="SELECT * FROM parquet.`%s`\n\tWHERE "%US_Weather_parquet+"\n\tor ".join(["measurement='%s'"%m for m in measurements])
print Query

SELECT * FROM parquet.`/US_Weather.parquet`
	WHERE measurement='TMAX'
	or measurement='TMIN'
	or measurement='TOBS'
	or measurement='SNOW'
	or measurement='SNWD'
	or measurement='PRCP'


In [20]:
t=time()
df = sqlContext.sql(Query).cache()
print df.count()
print 'took',time()-t,'seconds'

took 24.3902518749 seconds


In [21]:
print df.count()

4351091


In [37]:

t=time()

N=sc.defaultParallelism
print 'Number of executors=',N
rdd0=df.rdd.map(lambda row:(str(row['station']),((str(row['measurement'])\
                        ,row['year']),np.array([np.float64(row[str(i)]) for i in range(1,366)])))).cache()#.repartition(N)
print 'took',time()-t,'seconds'

Number of executors= 80
took 0.0116009712219 seconds


In [39]:
import numpy as np
def F(row):
    return (str(row['station']),((str(row['measurement'])\
                        ,row['year']),np.array([np.float64(row[str(i)]) for i in range(1,366)])))
row,=df.take(1)
#print F(row)

In [40]:
t=time()
print rdd0.count()
print 'took',time()-t,'seconds'

4351091
took 321.376906872 seconds


In [43]:
t=time()
print rdd0.count()
print 'took',time()-t,'seconds'

4351091
took 1.04327487946 seconds


In [44]:
t=time()
print rdd0.repartition(N).count()
print 'took',time()-t,'seconds'

4351091
took 5.36106610298 seconds


In [45]:
# Compute the overall distribution of values and the distribution of the number of nan per year
def find_percentiles(SortedVals,percentile):
  L=len(SortedVals)/percentile
  return SortedVals[L],SortedVals[-L]
  
def computeOverAllDist(rdd0):
  UnDef=np.array(rdd0.map(lambda row:sum(np.isnan(row))).sample(False,0.01).collect())
  flat=rdd0.flatMap(lambda v:list(v)).filter(lambda x: not np.isnan(x)).cache()
  count,S1,S2=flat.map(lambda x: np.float64([1,x,x**2]))\
                  .reduce(lambda x,y: x+y)
  mean=S1/count
  std=np.sqrt(S2/count-mean**2)
  Vals=flat.sample(False,0.0001).collect()
  SortedVals=np.array(sorted(Vals))
  low100,high100=find_percentiles(SortedVals,100)
  low1000,high1000=find_percentiles(SortedVals,1000)
  return {'UnDef':UnDef,\
          'mean':mean,\
          'std':std,\
          'SortedVals':SortedVals,\
          'low100':low100,\
          'high100':high100,\
          'low1000':low100,\
          'high1000':high1000
          }

In [ ]:
from numpy import linalg as LA

STAT={}  # dictionary storing the statistics for each measurement
Clean_Tables={}

for meas in measurements:
  t=time()
  Query="SELECT * FROM parquet.`%s`\n\tWHERE measurement = '%s'"%(US_Weather_parquet,meas)
  print Query
  df = sqlContext.sql(Query)
  rdd0=df.rdd.map(lambda row:(row['station'],((row['measurement'],row['year']),np.array([np.float64(row[str(i)]) for i in range(1,366)])))).cache()

  rdd1=rdd0.sample(False,1)\
           .map(lambda (key,val): val[1])\
           .cache()\
           .repartition(N)
  print rdd1.count()

  #get basic statistics
  STAT[meas]=computeOverAllDist(rdd1)   # Compute the statistics 
  low1000 = STAT[meas]['low1000']  # unpack the extreme values statistics
  high1000 = STAT[meas]['high1000']

  #clean up table from extreme values and from rows with too many undefinde entries.
  rdd2=rdd1.map(lambda V: np.array([x if (x>low1000-1) and (x<high1000+1) else np.nan for x in V]))
  rdd3=rdd2.filter(lambda row:sum(np.isnan(row))<50)
  Clean_Tables[meas]=rdd3.cache().repartition(N)
  C=Clean_Tables[meas].count()
  print 'for measurement %s, we get %d clean rows'%(meas,C)

  # compute covariance matrix
  OUT=computeCov(Clean_Tables[meas])

  #find PCA decomposition
  eigval,eigvec=LA.eig(OUT['Cov'])

  # collect all of the statistics in STAT[meas]
  STAT[meas]['eigval']=eigval
  STAT[meas]['eigvec']=eigvec
  STAT[meas].update(OUT)

  # print summary of statistics
  print 'the statistics for %s consists of:'%meas
  for key in STAT[meas].keys():
    e=STAT[meas][key]
    if type(e)==list:
      print key,'list',len(e)
    elif type(e)==np.ndarray:
      print key,'ndarray',e.shape
    elif type(e)==np.float64:
      print key,'scalar'
    else:
      print key,'Error type=',type(e)
  print 'time for',meas,'is',time()-t

SELECT * FROM parquet.`/US_Weather.parquet`
	WHERE measurement = 'TMAX'
662767
for measurement TMAX, we get 551661 clean rows
shape of S= (366, 366) shape of N= (366, 366)
shape of E= (365,) shape of NE= (365,)
the statistics for TMAX consists of:
E ndarray (365,)
Cov ndarray (365, 365)
O ndarray (365, 365)
NE ndarray (365,)
Var ndarray (365,)
SortedVals ndarray (22125,)
Mean ndarray (365,)
std scalar
UnDef ndarray (6689,)
eigval ndarray (365,)
high1000 scalar
NO ndarray (365, 365)
low100 scalar
high100 scalar
low1000 scalar
mean scalar
eigvec ndarray (365, 365)
time for TMAX is 218.555129051
SELECT * FROM parquet.`/US_Weather.parquet`
	WHERE measurement = 'TMIN'
662742
for measurement TMIN, we get 548833 clean rows
shape of S= (366, 366) shape of N= (366, 366)
shape of E= (365,) shape of NE= (365,)
the statistics for TMIN consists of:
E ndarray (365,)
Cov ndarray (365, 365)
O ndarray (365, 365)
NE ndarray (365,)
Var ndarray (365,)
SortedVals ndarray (22144,)
Mean ndarray (365,)
std sc

In [72]:
from pickle import dump
def dumpS3(object,S3dir,filename):
    dump(object,open(filename,'wb'))
    !ls -l $filename
    s3helper.local_to_s3(filename, S3dir+filename)
dumpS3((STAT,STAT_Descriptions),'/Weather/','STAT1.pickle')


-rw-rw-r-- 1 ec2-user ec2-user 81729695 Mar  9 06:09 STAT1.pickle


In [65]:
STAT.keys()

['TMIN', 'TOBS', 'TMAX', 'SNOW', 'SNWD', 'PRCP']

### Sample Stations
Generate a sample of stations, for each one store all available year X measurement pairs.

In [54]:
rdd0.take(10) # test output

[(u'USC00305618',
  ((u'PRCP', 1914.0),
   array([  nan,   nan,   nan,   nan,   nan,   nan,   nan,   nan,   nan,
            nan,   nan,   nan,   nan,   nan,   nan,   nan,   nan,   nan,
            nan,   nan,   nan,   nan,   nan,   nan,   nan,   nan,   nan,
            nan,   nan,   nan,   nan,   nan,   nan,   nan,   nan,   nan,
            nan,   nan,   nan,   nan,   nan,   nan,   nan,   nan,   nan,
            nan,   nan,   nan,   nan,   nan,   nan,   nan,   nan,   nan,
            nan,   nan,   nan,   nan,   nan,   nan,   nan,   nan,   nan,
            nan,   nan,   nan,   nan,   nan,   nan,   nan,   nan,   nan,
            nan,   nan,   nan,   nan,   nan,   nan,   nan,   nan,   nan,
            nan,   nan,   nan,   nan,   nan,   nan,   nan,   nan,   nan,
            nan,   nan,   nan,   nan,   nan,   nan,   nan,   nan,   nan,
            nan,   nan,   nan,   nan,   nan,   nan,   nan,   nan,   nan,
            nan,   nan,   nan,   nan,   nan,   nan,   nan,   nan,   nan,
           

In [73]:
groups=rdd0.groupByKey().cache()
print 'number of stations=',groups.count()

groups1=groups.sample(False,0.01).collect()
groups2=[(g[0],[e for e in g[1]]) for g in groups1]

dumpS3(groups2,'/Weather/','SampleStations.pickle')

number of stations= 45833
-rw-rw-r-- 1 ec2-user ec2-user 110449557 Mar  9 06:10 SampleStations.pickle


In [19]:
group_Sample=rdd0.sample(False,0.001).groupByKey().mapValues(list).cache()
group_Sample.first()

In [76]:
Means=rdd0.aggregateByKey((np.zeros(365),1),\
                          lambda S,D: sumWithNan(S,(D[1],1)),\
                          lambda S1,S2: sumWithNanithNan(S1,S2))\
.cache()#.repartition(N)

In [77]:
Means.take(5)

[(u'USC00213727',
  (array([[   67.,   511.,   326.,   285.,   122.,   321.,   193.,   155.,
             398.,   224.,   421.,   390.,   260.,   254.,   484.,   342.,
             249.,   124.,   133.,   194.,   232.,   228.,   267.,   546.,
             635.,   270.,    92.,   430.,   328.,   239.,   324.,    71.,
             171.,    73.,   244.,   421.,   133.,    54.,    81.,   218.,
             317.,   181.,    95.,    91.,   211.,   184.,   302.,   153.,
             175.,   244.,   188.,   272.,   115.,   330.,   452.,   159.,
             203.,   420.,   393.,   254.,   449.,   412.,   533.,   369.,
              51.,   315.,   125.,   317.,   113.,   176.,   730.,   215.,
             203.,   663.,   307.,   208.,   448.,   525.,   405.,   544.,
             310.,   295.,   365.,   206.,   585.,   391.,   570.,   331.,
             528.,   289.,   571.,   507.,   379.,   581.,   336.,   622.,
             530.,   473.,   899.,   645.,   242.,   407.,   683.,   547.,
       

In [78]:
groups=rdd0.groupByKey().cache()
print 'number of stations=',groups.count()


number of stations= 45833
